<a href="https://colab.research.google.com/github/MarcAssmann/morse_training/blob/main/train_morse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tensorflow Model to train Morse discriminator** 
22/05/20



In [ ]:
!git clone https://github.com/MarcAssmann/morse_training.git

Cloning into 'morse_training_data'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [37]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

(1) Process Data from Morse Generator

In [62]:
file = 'morse_training_data/platformio-device-monitor-210516-161118-cleaned.log'
pd.read_csv(file)

,R,G,B,Character
0,0,0,0,
1,39,38,30,
2,39,38,30,
3,39,38,30,
4,39,38,30,
...,...,...,...,...
79797,143,148,163,N
79798,143,130,102,N
79799,143,129,96,N
79800,144,149,163,N


In [63]:
data = pd.read_csv(file)
X = data[['B']]
y = data[['Character']]
y = y.replace(' ', '_')

In [76]:
# taken from https://stackoverflow.com/questions/43618245/how-to-one-hot-encode-sentences-at-the-character-level
def one_hot(c: str):
  idxs = np.fromstring(c, dtype=np.uint8) - ord('A')
  one_hot = tf.one_hot(idxs, 31, dtype=tf.uint8)
  one_hot = one_hot.numpy().tolist()[0]
  return one_hot

one_hot_mapping = {}

for i in range(ord('A'),ord('_')+1):
  c = chr(i)
  one_hot_mapping[c] = one_hot(c)

y_irgendwas = [ one_hot_mapping[c] for c in y['Character']]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [79]:
X_tensor = tf.convert_to_tensor(X)
y_tensor = tf.convert_to_tensor(y_irgendwas)
y_tensor

<tf.Tensor: shape=(79802, 31), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>

In [35]:
length = len(X_tensor)

X_train = X_tensor[:int(0.6*length)]
X_val = X_tensor[int(0.6*length):int(0.8*length)]
X_test = X_tensor[int(0.8*length):]

y_train = y_tensor[:int(0.6*length)]
y_val = y_tensor[int(0.6*length):int(0.8*length)]
y_test = y_tensor[int(0.8*length):]

(2) Create model

In [ ]:
model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
# model.add(layers.Embedding(input_dim=1000, output_dim=64))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(128))

# Add a Dense layer with 10 units.
model.add(layers.Dense(10))

model.summary()